In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import plotly.express as px
import nltk
import random

from dataset.util import *
%matplotlib inline

In [2]:
(conn, cur) = sql_connect()
articles = cur.execute('SELECT * FROM articles').fetchall()
texts = [a[4] for a in articles]
headlines = [a[2] for a in articles]
symbols = list(set([a[1] for a in articles]))
conn.close()
len(articles), articles[0]

(7401,
 (1,
  'AAPL',
  "Boomers and millennials both love Apple and Amazon, but here are the brands they don't agree on",
  '2020-03-07',
  "Millennials may not like labels in general, but they're as hooked on Amazon as boomers and Gen X.\n\n\nThat's according to the latest Brand Intimacy Survey by marketing and branding agency MBLM, which measures how emotionally connected Americans are to the brands they buy. Their scale is based on factors such as the positive feelings a user has for a company or label, and how much their values align, as well as the depth and intensity of the emotional connection in the brand relationship. And Visual Capitalist has made this data easily digestible by drawing up three charts that illustrate which age groups love which brands the most.\n\n\nAmazon AMZN, -1.19% was one of the few companies to be beloved across every age group, appearing in the top five for Gen Z/millennials, Gen X and baby boomers alike - and with boomers naming it their No. 1 brand 

In [3]:
(conn, cur) = sql_connect()
companies = cur.execute('SELECT * FROM companies').fetchall()
sym_to_comp = {}
for c in companies:
    sym_to_comp[c[1]] = c
conn.close()
len(companies), companies[0]

(81,
 (1,
  'AAPL',
  'Apple, Inc.',
  'Computers/Consumer Electronics',
  'Technology',
  'Apple, Inc. engages in the design, manufacture, and sale of smartphones, personal computers, tablets, wearables and accessories, and other variety of related services. It operates through the following geographical segments: Americas, Europe, Greater China, Japan, and Rest of Asia Pacific. The Americas segment includes North and South America. The Europe segment consists of European countries, as well as India, the Middle East, and Africa. The Greater China segment comprises of China, Hong Kong, and Taiwan. The Rest of Asia Pacific segment includes Australia and Asian countries. Its products and services include iPhone, Mac, iPad, AirPods, Apple TV, Apple Watch, Beats products, Apple Care, iCloud, digital content stores, streaming, and licensing services. The company was founded by Steven Paul Jobs, Ronald Gerald Wayne, and Stephen G. Wozniak on April 1, 1976 and is headquartered in Cupertino, C

In [ ]:
# StarSpace
# ./starspace train -trainFile input.txt -normalizeText false -model ssdocmodel -trainMode 1 -fileFormat labelDoc
# ./starspace test -testFile input2.txt -normalizeText false -basedoc input.txt -model ssdocmodel -thread 20 -trainMode 1 -verbose true -predictionFile input-out.txt
from collections import defaultdict
f = open('input.txt', 'a', encoding='utf-8')
a_by_s = defaultdict(list)
for a in articles:
    a_by_s[a[1]].append(a[4])
for k, v in a_by_s.items():
    t = [s.replace('\n\n\n', ' ') for s in v]
    t = [s.replace('\n', ' ') for s in t]
    t = [s.replace('\t', ' ') for s in t]
    t = [s.lower() for s in t]
    t = [' '.join(nltk.word_tokenize(s)) for s in t]
    f.write('\t'.join(t) + '\n')
f.close()

In [7]:
df = pd.DataFrame({
    'symb': [a[1] for a in articles],
    'sector': [sym_to_comp[a[1]][4] for a in articles],
    'headline': [a[2] for a in articles],
    'date': [a[3] for a in articles]
})
fig = px.histogram(df, x="date")

In [ ]:
# Doc2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
token_texts = [nltk.word_tokenize(s) for s in texts]
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
model = Doc2Vec(documents, vector_size=128, window=2, min_count=2, workers=4)

In [ ]:
X = np.array([model.infer_vector(s) for s in token_texts])

In [4]:
# Bert
# bert-serving-start -model_dir data/uncased_L-24_H-1024_A-16 -num_worker=1 -max_seq_len=NONE -max_batch_size 64
from bert_serving.client import BertClient
bc = BertClient()

In [5]:
X = bc.encode(headlines)

C:\Users\Shriv\Anaconda3\envs\tf-gpu\lib\site-packages\bert_serving\client\__init__.py:290: UserWarning:

server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency



In [33]:
X = bc.encode(texts)

C:\Users\Shriv\Anaconda3\envs\tf-gpu\lib\site-packages\bert_serving\client\__init__.py:290: UserWarning:

server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency



In [73]:
from sklearn.manifold import TSNE
import umap
reducer = umap.UMAP()
# tsne = TSNE(n_components=2)
X_embedded = reducer.fit_transform(X)

C:\Users\Shriv\Anaconda3\envs\tf-gpu\lib\site-packages\numba\typed_passes.py:293: NumbaPerformanceWarning:


The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "..\..\Anaconda3\envs\tf-gpu\lib\site-packages\umap\rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^


C:\Users\Shriv\Anaconda3\envs\tf-gpu\lib\site-packages\numba\typed_passes.py:293: NumbaPerformanceWarning:


The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "..\..\Anaconda3\envs\tf-gpu\lib\site-packages\umap\rp_tree.py", line 

In [74]:
df = pd.DataFrame({
    'x': X_embedded[:, 0], 'y': X_embedded[:, 1],
    'id': [a[0] for a in articles],
    'date': [a[3] for a in articles],
    'month': [a[3][:7] for a in articles],
    'symb': [a[1] for a in articles],
    'sector': [sym_to_comp[a[1]][4] for a in articles],
    'is_tech': ['Yes' if sym_to_comp[a[1]][4]=='Technology' else 'No' for a in articles],
    'virus': ['Yes' if ('covid' in a[4].lower() or 'corona' in a[4].lower()) else 'No' for a in articles],
    'headline': [a[2] for a in articles],
    'size': [1 for i, a in enumerate(articles)]
})
df.head()

,x,y,id,date,month,symb,sector,is_tech,virus,headline,size
0,8.150586,2.136733,1,2020-03-07,2020-03,AAPL,Technology,Yes,No,Boomers and millennials both love Apple and Am...,1
1,7.878256,0.242197,2,2020-03-07,2020-03,AAPL,Technology,Yes,Yes,Opinion: You've been diligently investing for ...,1
2,2.769109,-6.789793,3,2020-03-07,2020-03,AAPL,Technology,Yes,Yes,"Coronavirus outbreak aside, China stocks just ...",1
3,8.996351,4.483991,4,2020-03-07,2020-03,AAPL,Technology,Yes,Yes,"Coronavirus update: 101,733 cases, 3,460 death...",1
4,6.503841,0.498959,5,2020-03-07,2020-03,AAPL,Technology,Yes,Yes,"What Apple, Microsoft, GE and other U.S. compa...",1


In [77]:
fig = px.scatter(df, x="x", y="y", color="sector", hover_data=['sector', 'symb', 'headline'])
fig.show()

In [38]:
sym_to_idx = {s:symbols.index(s) for s in symbols}
sym_to_artid = {}
for s in symbols:
    arts = [a for a in articles if a[1] == s]
    art_idxs = [articles.index(a) for a in arts]
    random.shuffle(art_idxs)
    sym_to_artid[s] = art_idxs

A = []
AE = []
Y = []
for s in symbols:
    pos = sym_to_artid[s]
    n = len(pos)
    for i in range(n):
        A.append(sym_to_idx[s])
        AE.append(X[pos[i]])
        Y.append(1)
    for i in range(n):
        A.append(sym_to_idx[s])
        other_sym = s
        while other_sym == s:
            other_sym = random.choice(symbols)
        neg_ex = random.choice(sym_to_artid[other_sym])
        AE.append(X[neg_ex])
        Y.append(0)

A = np.array(A)
AE = np.array(AE)
Y = np.array(Y)
p = np.random.permutation(len(articles))
A = A[p]
AE = AE[p]
Y = Y[p]

In [42]:
from keras.layers import Input, Embedding, Dense, Dot, Reshape
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint 

input_symbol = Input(shape=(1,))
input_art_emb = Input(shape=(1024,))

embedding_symbol = Embedding(input_dim=len(symbols), output_dim=512)(input_symbol)
embedding_symbol = Reshape((512,))(embedding_symbol)

embedding_art = Dense(512, activation='sigmoid')(input_art_emb)

merged = Dot(normalize=True, axes=1)([embedding_symbol, embedding_art])
merged = Reshape(target_shape=(1,))(merged)

out = Dense(1, activation='sigmoid')(merged)
model = Model(inputs=[input_symbol, input_art_emb], outputs=out)

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
checkpoint = ModelCheckpoint('data/emb-model-{epoch:02d}-{val_accuracy:.2f}.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
hist = model.fit(x=[A, AE], y=Y, epochs=100, batch_size=16, validation_split=0.3, callbacks=[checkpoint])

train_df = pd.DataFrame(hist.history).reset_index()
loss_df = pd.melt(train_df, 
    id_vars="index", value_vars=['val_accuracy', 'accuracy'], value_name="loss")
fig = px.line(loss_df, x="index", y="loss", color="variable")
fig.show()


Train on 5180 samples, validate on 2221 samples
Epoch 1/100
5180/5180 [==============================] - 3s 533us/step - loss: 0.6645 - accuracy: 0.6454 - val_loss: 0.6312 - val_accuracy: 0.7479

Epoch 00001: val_accuracy improved from -inf to 0.74786, saving model to data/emb-model-01-0.75.h5
Epoch 2/100
5180/5180 [==============================] - 2s 480us/step - loss: 0.5976 - accuracy: 0.7761 - val_loss: 0.5726 - val_accuracy: 0.7893

Epoch 00002: val_accuracy improved from 0.74786 to 0.78928, saving model to data/emb-model-02-0.79.h5
Epoch 3/100
5180/5180 [==============================] - 2s 457us/step - loss: 0.5391 - accuracy: 0.8241 - val_loss: 0.5224 - val_accuracy: 0.8271

Epoch 00003: val_accuracy improved from 0.78928 to 0.82710, saving model to data/emb-model-03-0.83.h5
Epoch 4/100
5180/5180 [==============================] - 3s 502us/step - loss: 0.4927 - accuracy: 0.8479 - val_loss: 0.4938 - val_accuracy: 0.8271

Epoch 00004: val_accuracy did not improve from 0.82710
E


Epoch 00036: val_accuracy did not improve from 0.89599
Epoch 37/100
5180/5180 [==============================] - 3s 579us/step - loss: 0.0702 - accuracy: 0.9867 - val_loss: 0.2686 - val_accuracy: 0.8883

Epoch 00037: val_accuracy did not improve from 0.89599
Epoch 38/100
5180/5180 [==============================] - 3s 559us/step - loss: 0.0655 - accuracy: 0.9869 - val_loss: 0.2675 - val_accuracy: 0.8892

Epoch 00038: val_accuracy did not improve from 0.89599
Epoch 39/100
5180/5180 [==============================] - 3s 563us/step - loss: 0.0622 - accuracy: 0.9884 - val_loss: 0.2631 - val_accuracy: 0.8901

Epoch 00039: val_accuracy did not improve from 0.89599
Epoch 40/100
5180/5180 [==============================] - 3s 558us/step - loss: 0.0577 - accuracy: 0.9884 - val_loss: 0.2560 - val_accuracy: 0.8942

Epoch 00040: val_accuracy did not improve from 0.89599
Epoch 41/100
5180/5180 [==============================] - 3s 554us/step - loss: 0.0546 - accuracy: 0.9905 - val_loss: 0.2875 - v


Epoch 00076: val_accuracy did not improve from 0.89734
Epoch 77/100
5180/5180 [==============================] - 3s 564us/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.3771 - val_accuracy: 0.8897

Epoch 00077: val_accuracy did not improve from 0.89734
Epoch 78/100
5180/5180 [==============================] - 3s 558us/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.3872 - val_accuracy: 0.8847

Epoch 00078: val_accuracy did not improve from 0.89734
Epoch 79/100
5180/5180 [==============================] - 3s 601us/step - loss: 0.0263 - accuracy: 0.9921 - val_loss: 0.3459 - val_accuracy: 0.8843

Epoch 00079: val_accuracy did not improve from 0.89734
Epoch 80/100
5180/5180 [==============================] - 3s 585us/step - loss: 0.0260 - accuracy: 0.9934 - val_loss: 0.3543 - val_accuracy: 0.8937

Epoch 00080: val_accuracy did not improve from 0.89734
Epoch 81/100
5180/5180 [==============================] - 3s 577us/step - loss: 0.0066 - accuracy: 1.0000 - val_loss: 0.3721 - v

KeyboardInterrupt: 

In [46]:
model = load_model('data/emb-model-45-0.90.h5')
model.layers

In [48]:
sym_emb_model = Model(inputs=model.layers[0].get_input_at(0), outputs=model.layers[1].get_output_at(0))

In [71]:
import umap
reducer = umap.UMAP()

SX = sym_emb_model.predict(np.array(list(sym_to_idx.values()))).squeeze()
SX_embedded = reducer.fit_transform(SX)

In [72]:
dfs = pd.DataFrame({
    'x': SX_embedded[:, 0], 'y': SX_embedded[:, 1],
    'symb': list(sym_to_idx),
    'sector': [sym_to_comp[s][4] for s in sym_to_idx]
})
fig = px.scatter(dfs, x="x", y="y", color="sector", hover_name="symb")
fig.show()